In [ ]:
import pandas as pd
from scipy import stats
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
color_palette = {
 'CTL01A': '#DBB807',
    'CTL08A': '#0FB248',
    'CTL04E': '#FF0054',
    'CTL02A': '#7B00FF',
'H9': '#72190E',
 'H1': '#994F88',
 'CTL05A': '#1965B0',
 'CTL07C': '#437DBF',
 'CTL06F': '#CAE0AB',
 'CTL09A': '#FFFF00',
 'KTD8.2': '#E65518',
 'UCSFi001-A': '#7BAFDE'}

In [ ]:
CBO_d02 = pd.read_csv('./data/csv/CBO_day02_fit.csv', index_col=0)
CBO_d26 = pd.read_csv('./data/csv/CBO_day26_fit.csv', index_col=0)
iPSC_cum_growth = pd.read_csv('./data/csv/iPSC_fitted_exp_area_sum_per_split.csv', index_col=0)
census_d5 = pd.read_csv('./data/csv/CensusSeq_combined_weighted_rank_d5.csv', index_col=0)
census_d12 = pd.read_csv('./data/csv/CensusSeq_combined_weighted_rank_d12.csv', index_col=0)
census_d25 = pd.read_csv('./data/csv/CensusSeq_combined_weighted_rank_d25.csv', index_col=0)
census_d50 = pd.read_csv('./data/csv/CensusSeq_combined_weighted_rank_d50.csv', index_col=0)

In [ ]:
iPSC_cum_growth_mean = iPSC_cum_growth.groupby('Line')['rate'].mean().sort_index().rank()
iPSC_cum_growth_mean

In [ ]:
census_d5['combined_scores'].rank()

In [ ]:
ranks_df = pd.DataFrame({
    'Line': iPSC_cum_growth_mean.index,
    'iPSC_growth_rate': iPSC_cum_growth_mean.values,
    'CBO_velocity_0_2': CBO_d02['slope'].sort_index().rank().values,
    'CBO_velocity_2_6': CBO_d26['slope'].sort_index().rank().values,
    'CensusRank_d5': census_d5['combined_scores'].sort_index().rank().values,
    'CensusRank_d12': census_d12['combined_scores'].sort_index().rank().values,
    'CensusRank_d25': census_d25['combined_scores'].sort_index().rank().values,
    'CensusRank_d50': census_d50['combined_scores'].sort_index().rank().values,})

values_df = pd.DataFrame({
    'Line': iPSC_cum_growth_mean.index,
    'iPSC_growth_rate': iPSC_cum_growth.groupby('Line')['rate'].mean().sort_index().values,
    'CBO_velocity_0_2': CBO_d02['slope'].sort_index().values,
    'CBO_velocity_2_6': CBO_d26['slope'].sort_index().values,
    'CensusRank_d5': census_d5['combined_scores'].sort_index().values,
    'CensusRank_d12': census_d12['combined_scores'].sort_index().values,
    'CensusRank_d25': census_d25['combined_scores'].sort_index().values,
    'CensusRank_d50': census_d50['combined_scores'].sort_index().values,})

In [ ]:
ranks_df.set_index('Line', inplace=True)

In [ ]:
ranks_df_plot = pd.melt(ranks_df, ignore_index=False)
ranks_df_plot

In [ ]:
fig, ax = plt.subplots(figsize = (20, 5))
sns.lineplot(data = ranks_df_plot, x = 'variable', y = 'value', hue = 'Line', palette=color_palette)
ax.set_xlabel('Measurement', fontdict={'size': 20})
ax.set_ylabel('Rank', fontdict={'size': 20})
_ = ax.legend(bbox_to_anchor = (1,1))

In [ ]:
fig, ax = plt.subplots(figsize = (20, 5))
sns.lineplot(data = ranks_df_plot[~ranks_df_plot.variable.isin(['CBO_velocity_2_6'])], x = 'variable', y = 'value', hue = 'Line', palette=color_palette)
_ = ax.legend(bbox_to_anchor = (1,1))

In [ ]:
fig, ax = plt.subplots(figsize = (20, 5))
sns.lineplot(data = ranks_df_plot[~ranks_df_plot.variable.isin(['CBO_velocity_0_2', 'CBO_velocity_2_6'])], x = 'variable', y = 'value', hue = 'Line', palette=color_palette)
_ = ax.legend(bbox_to_anchor = (1,1))

In [ ]:
ranks_df_plot.variable.unique()

In [ ]:

keep = ['iPSC_growth_rate', 'CBO_velocity_0_2', 
       'CensusRank_d5']

fig, ax = plt.subplots(figsize = (20, 5))
sns.lineplot(data = ranks_df_plot[ranks_df_plot.variable.isin(keep)], x = 'variable', y = 'value', hue = 'Line', palette=color_palette)
_ = ax.legend(bbox_to_anchor = (1,1))

In [ ]:
values_df.columns = ['Line', 'hPSC growth rate', 'CBO growth rate - Day 0-2', 'CBO growth rate - Day 2-6',
                     'Census-Seq rank - Day 5', 'Census-Seq rank - Day 12', 'Census-Seq rank - Day 25', 'Census-Seq rank - Day 50']

In [ ]:
p_value < 0.0001

In [ ]:
p_value = pd.DataFrame(stats.spearmanr(values_df.drop('Line', axis  =1))[1], index = names, columns=names) 
p_value_copy = p_value.copy()
p_value_copy[p_value <= 0.05] = '*'
p_value_copy[p_value < 0.001] = '**'
p_value_copy[p_value < 0.0001] = '***'

p_value_copy[p_value > 0.05] = 'NS'

In [ ]:
cmap = sns.diverging_palette(230, 20, as_cmap=True)
mask = np.triu(np.ones_like(values_df.corr(), dtype=bool))

spearman = values_df.corr(method = 'spearman')


fig, ax = plt.subplots(figsize = (10, 10))
_ = sns.heatmap(spearman, cmap = cmap, mask = mask, square = True, 
                cbar_kws={'label': "Spearman's correlation coefficient", "shrink": 0.7}, annot = p_value_copy, fmt = '')
ax.figure.axes[-1].yaxis.label.set_size(15)
_ = ax.set_xticklabels(ax.get_xticklabels(), size = 15)
_ = ax.set_yticklabels(ax.get_yticklabels(), size = 15)
_ = ax.set_title("Correlation between growth parameters and Census-Seq ranking", fontdict={'size': 20})
plt.savefig('corr_matrix.svg', dpi = 300, bbox_inches = 'tight')

In [ ]:
fig, ax = plt.subplots(figsize = (10, 10))

names = ['hPSC growth rate', 'CBO growth rate - Day 0-2', 'CBO growth rate - Day 2-6',
                     'Census-Seq rank - Day 5', 'Census-Seq rank - Day 12', 'Census-Seq rank - Day 25', 'Census-Seq rank - Day 50']

p_value = pd.DataFrame(stats.spearmanr(values_df.drop('Line', axis  =1))[1], index = names, columns=names) 

mask = np.triu(np.ones_like(p_value, dtype=bool))
sns.heatmap(p_value, annot = True, mask = mask, square=True)